In [1]:
import numpy as np
import pandas as pd
import talib


import random
random.seed(42)


In [2]:
dataset = pd.read_csv('2330.TW.csv')
dataset = dataset.dropna()
dataset = dataset[['Open', 'High', 'Low', 'Close']]


dataset['H-L'] = dataset['High'] - dataset['Low']
dataset['O-C'] = dataset['Close'] - dataset['Open']
dataset['3day MA'] = dataset['Close'].shift(1).rolling(window = 3).mean()
dataset['10day MA'] = dataset['Close'].shift(1).rolling(window = 10).mean()
dataset['30day MA'] = dataset['Close'].shift(1).rolling(window = 30).mean()
dataset['Std_dev']= dataset['Close'].rolling(5).std()
dataset['RSI'] = talib.RSI(dataset['Close'].values, timeperiod = 9)
dataset['Williams %R'] = talib.WILLR(dataset['High'].values, dataset['Low'].values, dataset['Close'].values, 7)

dataset['Price_Rise'] = np.where(dataset['Close'].shift(-1) > dataset['Close'], 1, 0)

dataset = dataset.dropna()

X = dataset.iloc[:, 4:-1]
y = dataset.iloc[:, -1]

split = int(len(dataset)*0.8)
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

In [3]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

classifier = Sequential()
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu', input_dim = X.shape[1]))
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

dataset['y_pred'] = np.NaN
dataset.iloc[(len(dataset) - len(y_pred)):,-1:] = y_pred
trade_dataset = dataset.dropna()

trade_dataset['Tomorrows Returns'] = 0.
trade_dataset['Tomorrows Returns'] = np.log(trade_dataset['Close']/trade_dataset['Close'].shift(1))
trade_dataset['Tomorrows Returns'] = trade_dataset['Tomorrows Returns'].shift(-1)

trade_dataset['Strategy Returns'] = 0.
trade_dataset['Strategy Returns'] = np.where(trade_dataset['y_pred'] == True, trade_dataset['Tomorrows Returns'], - trade_dataset['Tomorrows Returns'])

trade_dataset['Cumulative Market Returns'] = np.cumsum(trade_dataset['Tomorrows Returns'])
trade_dataset['Cumulative Strategy Returns'] = np.cumsum(trade_dataset['Strategy Returns'])


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
170/170 [==============================] - 0s 802us/step - loss: 0.2502 - acc: 0.4471
Epoch 2/100
170/170 [==============================] - 0s 61us/step - loss: 0.2489 - acc: 0.6000
Epoch 3/100
170/170 [==============================] - 0s 65us/step - loss: 0.2478 - acc: 0.5412
Epoch 4/100
170/170 [==============================] - 0s 61us/step - loss: 0.2459 - acc: 0.5706
Epoch 5/100
170/170 [==============================] - 0s 64us/step - loss: 0.2434 - acc: 0.5588
Epoch 6/100
170/170 [==============================] - 0s 65us/step - loss: 0.2404 - acc: 0.6059
Epoch 7/100
170/170 [==============================] - 0s 69us/step - loss: 0.2379 - acc: 0.6000
Epoch 8/100
170/170 [==============================] - 0s 60us/step - loss: 0.2353 - acc: 0.6118
Epoch 9/100
170/170 [==============================] - 0s 65us/step - loss: 0.2370 - acc: 0.6059
Epoch 

170/170 [==============================] - 0s 61us/step - loss: 0.0592 - acc: 0.9412
Epoch 79/100
170/170 [==============================] - 0s 60us/step - loss: 0.0588 - acc: 0.9471
Epoch 80/100
170/170 [==============================] - 0s 63us/step - loss: 0.0542 - acc: 0.9529
Epoch 81/100
170/170 [==============================] - 0s 58us/step - loss: 0.0524 - acc: 0.9471
Epoch 82/100
170/170 [==============================] - 0s 64us/step - loss: 0.0514 - acc: 0.9529
Epoch 83/100
170/170 [==============================] - 0s 67us/step - loss: 0.0524 - acc: 0.9529
Epoch 84/100
170/170 [==============================] - 0s 60us/step - loss: 0.0490 - acc: 0.9588
Epoch 85/100
170/170 [==============================] - 0s 61us/step - loss: 0.0502 - acc: 0.9588
Epoch 86/100
170/170 [==============================] - 0s 61us/step - loss: 0.0505 - acc: 0.9588
Epoch 87/100
170/170 [==============================] - 0s 58us/step - loss: 0.0514 - acc: 0.9529
Epoch 88/100
170/170 [===========

/home/gtx1080/.local/lib/python3.5/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/gtx1080/.local/lib/python3.5/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/gtx1080/.local/lib/python3.5/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

In [4]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
plt.plot(trade_dataset['Cumulative Market Returns'], color='r', label='Market Returns')
plt.plot(trade_dataset['Cumulative Strategy Returns'], color='g', label='Strategy Returns')
plt.legend()
plt.show()

<Figure size 1000x500 with 1 Axes>